In [36]:
import numpy as np
import pandas as pd

In [37]:
# Load in each dataset
data21 = pd.read_excel("Data/2021 Enrollment Demographics.xlsx", sheet_name="By School")
data22 = pd.read_excel("Data/2022 Enrollment Demographics.xlsx", sheet_name="By School")
data23 = pd.read_excel("Data/2023 Enrollment Demographics.xlsx", sheet_name="By School")

In [38]:
nameIndex = pd.read_excel("Data/Another School List.xlsx", sheet_name="School")
# Some schools have closed, so it is necessary to include them
closed = pd.read_excel("Data/Another School List.xlsx", sheet_name="Closed Schools")

# The CompositeSchoolID is composed of the LEA ID, the School ID, and the School Number
nameIndex["CompositeSchoolID"] = nameIndex["LEA ID"].astype(str) + nameIndex["School ID"].astype(str) + nameIndex["School Number"].astype(str)
closed["CompositeSchoolID"] = closed["LEA ID"].astype(str) + closed["School ID"].astype(str) + closed["School Number"].astype(str)
nameIndex = nameIndex[["LEA Name", "School Name", "CompositeSchoolID"]]
closed = closed[["LEA Name", "School Name", "CompositeSchoolID"]]

nameIndex = pd.concat([nameIndex, closed])

# There are a few leftover schools after matching. The dataset will be altered to match those
changeDict = {
    "Cache Alternative High": "Cache High",
    "Big Water School": "Big Water High",
    "Pleasant Creek School": "Pleasant Creek High School",
    "Ashley Valley Education Ctr.": "Ashley Valley Educ. Ctr.",
    "InTech Collegiate Academy": "Intech Collegiate Academy",
    "No. UT. Acad. of Math Engineering & Science - Ogden": "No. UT. Acad. of Math Engineering & Science Ogden",
    "Salt Lake Academy High School": "Real Salt Lake Academy High School",
    "Spectrum Academy": "Spectrum Academy - NSL",
    "Success Academy": "Success SUU"
}
nameIndex["School Name"] = nameIndex["School Name"].replace(changeDict)

del nameIndex["LEA Name"]

nameIndex

,School Name,CompositeSchoolID
0,A to Z Building Blocks American Fork on 100 East,186651186697901
1,A to Z Building Blocks American Fork on 300 East,186651186698902
2,A to Z Building Blocks Draper,186651186699903
3,A to Z Building Blocks in Spanish Fork,186651186727906
4,A to Z Building Blocks on Ranches Parkway,186651186700904
...,...,...
423,Weber State University Charter Academy,186314186315100
424,Whitmore Academy,145604145604951
425,Willow Creek School,186003186003952
426,Winter Sports School,186187186187999


In [39]:
def CleanEnrollmentDemographics(data):
    """Cleans the dataset"""
    """Parameters: data (pandas dataframe)"""
    """Returns: cleaned (pandas dataframe)"""
    cleaned = data.copy()
    
    # Drop all the non-high schools.
    cleaned = cleaned[cleaned['Grade_12'] > 0]
    cleaned = cleaned[cleaned['Grade_11'] > 0]
    
    # Drop irrelevant columns
    irrelevantColumns = ["Grade_" + str(i) for i in range(1,9)] # I'm choosing to include grade 9, for now.
    irrelevantColumns += ["K", "Preschool"]
    cleaned = cleaned.drop(irrelevantColumns, axis="columns")
    
    # Merge to add the composite school id
    cleaned = cleaned.merge(nameIndex, how="left", on="School Name")
    
    # Drop the double-up LEA columns
    
    return cleaned

In [40]:
clean21 = CleanEnrollmentDemographics(data21)

In [41]:
clean22 = CleanEnrollmentDemographics(data22)

In [42]:
clean23 = CleanEnrollmentDemographics(data23)

In [43]:
fullData = pd.concat([clean21, clean22, clean23])
fullData.to_csv('Cleaned Datasets/EnrollmentDemographics_Cleaned.csv')